In [1]:
!unzip open.zip

Archive:  open.zip
  inflating: sample_submission.csv   
  inflating: test_data.csv           
  inflating: topic_dict.csv          
  inflating: train_data.csv          


In [7]:
pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 450kB 32.9MB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


# DAY2(21/07/02)
## 데이콘 자연어 처리 첫 컴피티션 참가하기
### 링크 : https://dacon.io/competitions/official/235747/overview/description
#### 1. 컴피티션 주제 : 뉴스 토픽 분류 AI 경진대회 - 한국어 뉴스 헤드라인을 이용하여 뉴스의 주제를 분류하는 알고리즘 개발
#### 2. DATA : train_data(index / title / topic_idx), test_data(index / title ), topic_dict(topic_idx)
#### 3. 목표 : tensorflow LSTM 모델 구현을 통해 컴피티션에 제출하는 것.



In [372]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
import konlpy
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer

In [373]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
submission = pd.read_csv('sample_submission.csv')
topic = pd.read_csv('topic_dict.csv')

In [398]:
train

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


### 1. 텍스트 간단한 전처리

### 1-1. 정규 표현식을 바탕으로 텍스트 전처리

In [374]:
def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent) ##정규표현식으로 텍스트 전처리 진행
    return sent_clean 

In [375]:
y_train = train['topic_idx']
X_train = train['title'].apply(clean_text)
X_test = test['title'].apply(clean_text)
X_train.shape, y_train.shape, X_test.shape

((45654,), (45654,), (9131,))

In [376]:
X_test = test['title'].apply(clean_text)

### 1-2. Konlpy를 활용한 명사만을 추출하는 형태의 텍스트 전처리

In [399]:
okt = Okt()

train_token = train['title'].apply(okt.nouns) #텍스트 전처리 - 명사만 추출
test_token = test['title'].apply(okt.nouns) #텍스트 전처리 - 명사만 추출

In [406]:
train_token = train_token.apply(lambda x : ' '.join(x))
test_token = test_token.apply(lambda x : ' '.join(x))

In [408]:
train_token, test_token

(0              인천 핀란드 항공기 결항 휴가 철 여행객 분통
 1                         실리콘밸리 구글 전역 거점
 2            이란 외무 긴장 완화 해결 책 미국 경제 전쟁 것
 3             클린턴 측근 기업 특수 관계 조명 공과 사 종합
 4              시진핑 트럼프 중미 무역 협상 조속 타결 희망
                       ...               
 45649             금융 미국 스티펠 제휴 선진국 시장 공략
 45650    보 서울시 교육청 신종 코로나 확산 개학 연기 휴업 검토
 45651               게시판 증권 영웅 전 실전 투자 대회
 45652                  답변 배 기동 국립 중앙 박물관
 45653        한국 인터넷 기자 시상식 내달 개최 특별상 김성후
 Name: title, Length: 45654, dtype: object,
 0                  유튜브 내달 크리에이터 지원 공간 운영
 1                          어버이날 남부 지방 황사
 2                    내년 국가 평가 때 논문 건수 반영
 3               김명자 신임 과총 회장 원로 과학자 지혜 것
 4       회색 인간 작가 김 동식 양심 고백 등 새 소설 집 권 간
                       ...               
 9126                       인천 오후 대설주의보 눈
 9127        노래방 지인 성추행 외교부 사무관 불구속 입건 종합
 9128             전 부마항쟁 부산 시위 사진 점 최초 공개
 9129        게시판 아리랑 아프리카 개발 은행 총회 개회 생중계
 9130      유영민 과 장관 강소특구 지역 혁신 중심 지원 책 강구
 Name: title, Length: 9131, dtyp

### 2. 텍스트 벡터화 진행

### 2-1. 정규 표현식으로 전처리한 텍스트 벡터화

In [377]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
sequences = pad_sequences(sequences, 13)
sequences

array([[    0,     0,     0, ...,  4338,  2967,  9824],
       [    0,     0,     0, ...,  8107,  1440, 27221],
       [    0,     0,     0, ..., 12391, 16840,    17],
       ...,
       [    0,     0,     0, ...,  7492, 76524,  9789],
       [    0,     0,     0, ...,  3413, 76525, 76526],
       [    0,     0,     0, ...,    24, 14059, 76528]], dtype=int32)

In [378]:
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), 13)
X_test

array([[    0,     0,     0, ...,    51,  4031,   274],
       [    0,     0,     0, ...,  4283, 17822,  3073],
       [    0,     0,     0, ...,   250,  2358,  4795],
       ...,
       [    0,     0,     0, ...,    73,   395,    42],
       [    0,     0,     0, ...,  3206,  5282,  1215],
       [    0,     0,     0, ...,  1133, 15050,  4718]], dtype=int32)

In [380]:
word_index = tokenizer.word_index
word_index

{'일': 1,
 '대통령': 2,
 '명': 3,
 '년': 4,
 '첫': 5,
 '월': 6,
 '억원': 7,
 '등': 8,
 '보': 9,
 '게시판': 10,
 '출시': 11,
 '신간': 12,
 '감독': 13,
 '천': 14,
 '이란': 15,
 '위': 16,
 '것': 17,
 '분기': 18,
 '대': 19,
 '트럼프': 20,
 '한국': 21,
 '개': 22,
 '개발': 23,
 '개최': 24,
 '전': 25,
 '만에': 26,
 '정부': 27,
 '중': 28,
 '경기': 29,
 '삼성': 30,
 '만': 31,
 '올해': 32,
 '코로나': 33,
 '서울': 34,
 '작년': 35,
 '월드컵': 36,
 '김정은': 37,
 '그래픽': 38,
 '도': 39,
 '문': 40,
 '새': 41,
 '공개': 42,
 '더': 43,
 '시즌': 44,
 '류현진': 45,
 '최대': 46,
 '연승': 47,
 '중국': 48,
 '제': 49,
 '네이버': 50,
 '지원': 51,
 '억': 52,
 '코스피': 53,
 '연속': 54,
 '국내': 55,
 '사망': 56,
 '에': 57,
 '미국': 58,
 '또': 59,
 '개막': 60,
 '서비스': 61,
 '추진': 62,
 '내년': 63,
 '한': 64,
 '터키': 65,
 '전국': 66,
 '조': 67,
 '꺾고': 68,
 '갤럭시': 69,
 '차': 70,
 '듯': 71,
 '세계': 72,
 '점': 73,
 '최고': 74,
 '주말': 75,
 '여행': 76,
 '프로농구': 77,
 '특징주': 78,
 '영업익': 79,
 '시': 80,
 '총리': 81,
 '후': 82,
 '확대': 83,
 '증가': 84,
 '북한': 85,
 '가능성': 86,
 '아시안게임': 87,
 '논란': 88,
 '속': 89,
 '회': 90,
 '투자': 91,
 '내달': 92,
 '주년': 93

### 2-2. Konlpy를 통한 명사 추출 텍스트 전처리 벡터화

In [410]:
token = Tokenizer()
token.fit_on_texts(train_token)
seq = tokenizer.texts_to_sequences(train_token)
train_x = pad_sequences(seq, 13)
train_x

array([[    0,     0,     0, ...,  2739,  2967,  9824],
       [    0,     0,     0, ...,   184,  1440,  4130],
       [    0,     0,     0, ...,   411,   742,    17],
       ...,
       [    0,     0,     0, ...,  2682,    91,   855],
       [    0,     0,     0, ...,  9558,  7546,  6040],
       [    0,     0,     0, ...,    24, 14059, 76528]], dtype=int32)

In [411]:
seq = tokenizer.texts_to_sequences(test_token)
test_x = pad_sequences(seq, 13)
test_x

array([[    0,     0,     0, ...,    51,  4031,   274],
       [    0,     0,     0, ...,  1201,  2608,  3073],
       [    0,     0,     0, ...,  4198,  6654,  2358],
       ...,
       [    0,     0,     0, ...,    73,   395,    42],
       [    0,     0,     0, ...,  3206, 11017,  1215],
       [    0,     0,     0, ...,    51,   712,  4718]], dtype=int32)

In [379]:
len(X_test[0])

13

In [412]:
token.word_index

{'종합': 1,
 '대통령': 2,
 '한국': 3,
 '명': 4,
 '위': 5,
 '첫': 6,
 '삼성': 7,
 '전': 8,
 '등': 9,
 '보': 10,
 '이란': 11,
 '감독': 12,
 '출시': 13,
 '경기': 14,
 '게시판': 15,
 '트럼프': 16,
 '신간': 17,
 '것': 18,
 '중': 19,
 '정부': 20,
 '투자': 21,
 '개발': 22,
 '개': 23,
 '서울': 24,
 '지원': 25,
 '제': 26,
 '더': 27,
 '최고': 28,
 '주': 29,
 '중국': 30,
 '올해': 31,
 '민주': 32,
 '개최': 33,
 '차': 34,
 '영업': 35,
 '회': 36,
 '세계': 37,
 '현대': 38,
 '비': 39,
 '미국': 40,
 '공개': 41,
 '최': 42,
 '기술': 43,
 '기업': 44,
 '안': 45,
 '증권': 46,
 '코로나': 47,
 '도': 48,
 '작년': 49,
 '대표': 50,
 '금융': 51,
 '시장': 52,
 '게임': 53,
 '최대': 54,
 '김정은': 55,
 '총리': 56,
 '북한': 57,
 '선': 58,
 '문': 59,
 '서비스': 60,
 '사망': 61,
 '전자': 62,
 '축제': 63,
 '국내': 64,
 '달': 65,
 '사업': 66,
 '류현진': 67,
 '새': 68,
 '터키': 69,
 '전국': 70,
 '합의': 71,
 '점': 72,
 '후': 73,
 '그래픽': 74,
 '여행': 75,
 '추진': 76,
 '월드컵': 77,
 '팀': 78,
 '대회': 79,
 '네이버': 80,
 '장관': 81,
 '내년': 82,
 '방문': 83,
 '부산': 84,
 '연승': 85,
 '회의': 86,
 '시즌': 87,
 '축구': 88,
 '사': 89,
 '제재': 90,
 '연속': 91,
 '승': 92,
 '갤럭시': 93,
 '

In [381]:
len(sequences[4])

13

#### 대략적인 것을 볼때, 명사만을 추출한 것이 의미적으로 더 유의미해보인다. 그렇다면 명사만을 추출한 것을 우선적으로 시퀀스 모델로 학습을 진행하겠음.

### 3. Train / Validation 분리 진행

In [382]:
X_train, X_val, y_train, y_val = train_test_split(sequences, np.array(train['topic_idx']), test_size = 0.2, random_state =411)

In [383]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((36523, 13), (9131, 13), (36523,), (9131,))

### 3-2. 명사만을 추출하여 전처리 한 데이터 Train / Validation 분리

In [413]:
train_x, val_x, train_y, val_y = train_test_split(train_x, np.array(train['topic_idx']), test_size = 0.2, random_state =411)

In [414]:
train_x.shape, val_x.shape, val_y.shape, train_y.shape

((36523, 13), (9131, 13), (9131,), (36523,))

### 4. Sequential 모델로 LSTM 생성

In [420]:
## 하이퍼 파라미터 세팅
batch_size = 128
num_epochs = 20
vocab_size = len(word_index)+1
emb_size = 128
hidden_dimension = 256
output_dimension = 7

In [421]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, emb_size))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.LSTM(256, return_sequences=True))
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dense(hidden_dimension, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(output_dimension, activation = 'softmax'))

In [422]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)

model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name = 'accuracy')])

In [423]:
model.fit(train_x, train_y, epochs=num_epochs, batch_size=batch_size, callbacks=[es], validation_data=(val_x, val_y))

Epoch 1/20
286/286 [==============================] - 110s 364ms/step - loss: 1.8212 - accuracy: 0.8381 - val_loss: 1.4927 - val_accuracy: 0.9456
Epoch 2/20
286/286 [==============================] - 102s 356ms/step - loss: 1.1832 - accuracy: 0.9662 - val_loss: 0.9188 - val_accuracy: 0.9769
Epoch 3/20
286/286 [==============================] - 101s 355ms/step - loss: 0.7905 - accuracy: 0.9845 - val_loss: 0.7238 - val_accuracy: 0.9857
Epoch 4/20
286/286 [==============================] - 103s 359ms/step - loss: 0.6386 - accuracy: 0.9898 - val_loss: 0.6776 - val_accuracy: 0.9874
Epoch 5/20
286/286 [==============================] - 103s 360ms/step - loss: 0.5575 - accuracy: 0.9926 - val_loss: 0.6520 - val_accuracy: 0.9892
Epoch 6/20
286/286 [==============================] - 103s 359ms/step - loss: 0.5038 - accuracy: 0.9939 - val_loss: 0.6402 - val_accuracy: 0.9892
Epoch 7/20
286/286 [==============================] - 103s 360ms/step - loss: 0.4606 - accuracy: 0.9947 - val_loss: 0.6332 -

#### 사실 이 모델을 쓰기 전에, 정규 표현식으로 먼저 진행을 했었고, 추가적인 튜닝을 한 후 명사만을 추출한 텍스트 데이터로 다시 학습을 진행하였다. 
#### 정규 표현식의 경우, 검증 데이터가 약 97%를 맴돌았지만 명사를 이용한 경우, 98%까지 올라가며 1%정도 성능이 향상된 것을 볼 수 있었다.

### 5. Subclassing 모델로 LSTM 생성

In [252]:
model_name = 'rnn_classifier'
batch_size = 64
num_epochs = 15

kargs = {'model_name' : model_name,
         'vocab_size' : len(word_index)+1,
         'embedding_dimension' : 128,
         'dropout_rate' : 0.2,
         'lstm_dimension' : 256,
         'dense_dimension' : 256,
         'output_dimension' : 7}



In [253]:
class RNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = tf.keras.layers.Embedding(input_dim = kargs['vocab_size'], output_dim=kargs['embedding_dimension'])
        self.lstm_1_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'], return_sequences = True)
        self.lstm_2_later = tf.keras.layers.LSTM(kargs['lstm_dimension'])
        self.dropout = tf.keras.layers.Dropout(kargs['dropout_rate'])
        self.fc1 = tf.keras.layers.Dense(units=kargs['dense_dimension'], activation = tf.keras.activations.relu)
        self.fc2 = tf.keras.layers.Dense(units=kargs['output_dimension'],activation = tf.keras.activations.softmax)

    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.lstm_1_layer(x)
        x = self.lstm_2_later(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x


lstm_model = RNNClassifier(**kargs)
lstm_model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name = 'accuracy')])

In [254]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
history = lstm_model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size,  validation_data=(X_val,y_val), callbacks=[es])

Epoch 1/15
571/571 [==============================] - 160s 271ms/step - loss: 1.6448 - accuracy: 0.8563 - val_loss: 1.1436 - val_accuracy: 0.9569
Epoch 2/15
571/571 [==============================] - 153s 268ms/step - loss: 0.9296 - accuracy: 0.9746 - val_loss: 0.9196 - val_accuracy: 0.9710
Epoch 3/15
571/571 [==============================] - 153s 267ms/step - loss: 0.6117 - accuracy: 0.9912 - val_loss: 0.8617 - val_accuracy: 0.9751
Epoch 4/15
571/571 [==============================] - 154s 269ms/step - loss: 0.4199 - accuracy: 0.9956 - val_loss: 0.9288 - val_accuracy: 0.9749
Epoch 5/15
571/571 [==============================] - 153s 267ms/step - loss: 0.2961 - accuracy: 0.9973 - val_loss: 1.0045 - val_accuracy: 0.9755
Epoch 6/15
571/571 [==============================] - 154s 270ms/step - loss: 0.2174 - accuracy: 0.9985 - val_loss: 1.1118 - val_accuracy: 0.9729
Epoch 7/15
571/571 [==============================] - 154s 270ms/step - loss: 0.1638 - accuracy: 0.9991 - val_loss: 1.1774 -

#### 정규 표현식으로만 전처리한 것을 학습한 결과, 97%의 성능을 나온 것을 볼 수 있다.

### 6. 테스트 예측값 생성 및 데이콘 제출

In [424]:
tmp_pred = model.predict(X_test)
pred = np.argmax(tmp_pred, axis = 1)
pred

array([3, 3, 2, ..., 2, 2, 2])

In [ ]:
submission['topic_idx'] = pred
submission.to_csv('baseline7.csv', index = False)
submission

In [304]:
!pip install dacon_submit_api-0.0.4-py3-none-any.whl

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl


In [427]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/content/baseline7.csv', 
'안알려줌', 
'235747', 
'안알려줌', 
'memo' )

{'isSubmitted': True, 'detail': 'Success'}


### 7. 느낀점
#### 현재 LSTM 모델을 만들기 이전에 내가 여태껏 알던 방식으로 막 쌓아보았다. 그랬을 때, 검증 성능이 80%에서 더이상 오르지 않았다. 그래서 텍스트의 전처리 문제거나 모델 성능이 안좋게 나오는 모델이라고만 생각을 했었다.
#### 하지만, 현재 공부하는 내용을 잘 정리해놓은 노트를 보고 그저께 배웠던 방식과 결합하여 적용했는데, 성능이 말도 안되게 오르는 것을 볼 수 있었다.
#### 내가 여태껏 알던 방식은 LSTM을 잘 활용하지 못했다. 아니, 레이어를 제대로 쌓지 못했다는 것을 느낄 수 있었다. 그래도 이번 기회에 알아갈 수 있게 되어 정말 다행이라고 생각한다.
#### 검증셋의 성능이 매우 좋았기 때문에, 컴피티션 제출했을 때, 높은 순위를 달성할 수 있을 줄 알았다. 하지만, 현재 제출한 사람들 중 하위권에 맴도는 것을 보고 약간 충격을 받았다.
#### 왜냐하면 내가 잘했기 때문이라기 보다는, 검증 셋에서 성능이 너무 잘 나왔기 때문에, 테스트 데이터 또한 잘 예측할 수 있을 것이라 생각했다. 하지만 그렇지 않았고 내가 잘못한 부분이 있는건지 다시 한번 혼란스러워졌다.
#### 검색을 하다보니 컴피티션의 경우, 퍼블릭과 프라이빗이 있는데 퍼블릭은 50%를 예측하고 프라이빗은 대회가 끝나고 나머지 50%와 결합하여 다시 한번 스코어를 내기 때문에, 현재 나온 점수가 정확한 점수라고 볼수는 없다. 그래도... 1등과 15%의 성능 차이가 난다는 것이.. 꽤나 찝찝하긴 하다..